***GENERATED CODE FOR academicsuccessregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Target', 'transformation_label': 'String Indexer'}], 'feature': 'Target', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Dropout', 'max': 'Graduate', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Target'}, {'feature_label': 'Target', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Target')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run academicsuccessregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	highereduprediction = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Higher Edu Prediction.csv', 'filename': 'Higher Edu Prediction.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(highereduprediction)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run academicsuccessregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	academicsuccessregressionautofe = TransformationMain.run(highereduprediction,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Marital status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.11", "stddev": "0.46", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "Marital status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Application mode", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.33", "stddev": "5.29", "min": "1", "max": "17", "missing": "0"}, "updatedLabel": "Application mode"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Application order", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.79", "stddev": "1.36", "min": "1", "max": "6", "missing": "0"}, "updatedLabel": "Application order"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Course", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "10.36", "stddev": "4.02", "min": "1", "max": "17", "missing": "0"}, "updatedLabel": "Course"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Daytime/evening attendance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.91", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Daytime/evening attendanc..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Previous qualification", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.44", "stddev": "3.93", "min": "1", "max": "17", "missing": "0"}, "updatedLabel": "Previous qualification"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Nacionality", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.11", "stddev": "1.14", "min": "1", "max": "15", "missing": "0"}, "updatedLabel": "Nacionality"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Mother's qualification", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "12.05", "stddev": "8.83", "min": "1", "max": "27", "missing": "0"}, "updatedLabel": "Mother's qualification"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Father's qualification", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "16.8", "stddev": "10.79", "min": "1", "max": "29", "missing": "0"}, "updatedLabel": "Father's qualification"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Mother's occupation", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.15", "stddev": "3.52", "min": "1", "max": "29", "missing": "0"}, "updatedLabel": "Mother's occupation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Father's occupation", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.6", "stddev": "4.32", "min": "1", "max": "46", "missing": "0"}, "updatedLabel": "Father's occupation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Displaced", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.55", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Displaced"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Educational special needs", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.12", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Educational special needs"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Debtor", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.09", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Debtor"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Tuition fees up to date", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.93", "stddev": "0.26", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Tuition fees up to date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Gender", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.08", "stddev": "0.27", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Scholarship holder", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.28", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Scholarship holder"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age at enrollment", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "21.93", "stddev": "6.11", "min": "18", "max": "55", "missing": "0"}, "updatedLabel": "Age at enrollment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "International", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "International"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 1st sem (credited)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.51", "stddev": "1.9", "min": "0", "max": "19", "missing": "0"}, "updatedLabel": "Curricular units 1st sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 1st sem (enrolled)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.24", "stddev": "2.02", "min": "0", "max": "21", "missing": "0"}, "updatedLabel": "Curricular units 1st sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 1st sem (evaluations)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "8.08", "stddev": "3.5", "min": "0", "max": "24", "missing": "0"}, "updatedLabel": "Curricular units 1st sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 1st sem (approved)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.96", "stddev": "2.62", "min": "0", "max": "21", "missing": "0"}, "updatedLabel": "Curricular units 1st sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 1st sem (grade)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.13", "stddev": "4.25", "min": "0.0", "max": "17.125", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Curricular units 1st sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 1st sem (without evaluations)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.08", "stddev": "0.41", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "Curricular units 1st sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 2nd sem (credited)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.35", "stddev": "1.41", "min": "0", "max": "13", "missing": "0"}, "updatedLabel": "Curricular units 2nd sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 2nd sem (enrolled)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.31", "stddev": "1.82", "min": "0", "max": "19", "missing": "0"}, "updatedLabel": "Curricular units 2nd sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 2nd sem (evaluations)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.93", "stddev": "3.56", "min": "0", "max": "19", "missing": "0"}, "updatedLabel": "Curricular units 2nd sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 2nd sem (approved)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "4.71", "stddev": "2.74", "min": "0", "max": "19", "missing": "0"}, "updatedLabel": "Curricular units 2nd sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 2nd sem (grade)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.82", "stddev": "4.79", "min": "0.0", "max": "17.5875", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Curricular units 2nd sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Curricular units 2nd sem (without evaluations)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.11", "stddev": "0.56", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "Curricular units 2nd sem ..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Unemployment rate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "11.6", "stddev": "2.73", "min": "7.6", "max": "16.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Unemployment rate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Inflation rate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.22", "stddev": "1.37", "min": "-0.8", "max": "3.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Inflation rate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "GDP", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "2.22", "min": "-4.06", "max": "3.51", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "GDP"}, {"transformationsData": [{"feature_label": "Target", "transformation_label": "String Indexer"}], "feature": "Target", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Dropout", "max": "Graduate", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Target"}]}))

	#transformationPostExecutionHook(academicsuccessregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run academicsuccessregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(academicsuccessregressionautofe, ["Marital status", "Application mode", "Application order", "Course", "Daytime/evening attendance", "Previous qualification", "Nacionality", "Mother's qualification", "Father's qualification", "Mother's occupation", "Father's occupation", "Displaced", "Educational special needs", "Debtor", "Tuition fees up to date", "Gender", "Scholarship holder", "Age at enrollment", "International", "Curricular units 1st sem (credited)", "Curricular units 1st sem (enrolled)", "Curricular units 1st sem (evaluations)", "Curricular units 1st sem (approved)", "Curricular units 1st sem (grade)", "Curricular units 1st sem (without evaluations)", "Curricular units 2nd sem (credited)", "Curricular units 2nd sem (enrolled)", "Curricular units 2nd sem (evaluations)", "Curricular units 2nd sem (approved)", "Curricular units 2nd sem (grade)", "Curricular units 2nd sem (without evaluations)", "Unemployment rate", "Inflation rate", "GDP"], "Target_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

